In [ ]:
import pandas as pd
import os
import math

#Sets df equal to amazon prime movies.csv
df = pd.read_csv('/content/gdrive/My Drive/Colab Datasets/amazon prime movies.csv')
print(df)

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
               Movie Name  ...                                               Plot
0              John Rambo  ...  In Thailand, John Rambo joins a group of merce...
1            American Pie  ...  You'll never look at warm apple pie the same w...
2               Bombshell  ...  The provocative real story of three smart, amb...
3              Love Birds  ...  After accidentally meeting, Mridula (Nagma) an...
4                   Hippi  ...  Hippi is about the live-in relationship betwee...
...                   ...  ...                                                ...
8122             Blackout  ...  This ensemble thriller tells the story of four...
8123               Loafer  ...  Loafer Is A Bollywood Romantic Drama Hindi Mov...
8124          Remember Me  ...  Two people scarred by grief find happiness in ...
8125                Khopa  ...  Narya who comes f

In [ ]:
#Drop the plot column
df_clean = df.drop(['Plot'], axis=1, inplace=True)
#Drop all column with "None" 
df_clean = df.dropna(subset = ['IMDb Rating','Year of Release','Language', 'Running Time', 'Maturity Rating'])
df_clean = df_clean.loc[(df_clean['IMDb Rating'] != "None") & (df_clean['Year of Release'] != "None") & (df_clean['Language'] != "None") & (df_clean['Running Time'] != "None") & (df_clean['Maturity Rating'] != "None")]
#Drops some specific values that were anomalies in this data set 
df_clean.drop(df_clean[df_clean["Movie Name"] == "Kasu Mela Kasu"].index, inplace=True)
df_clean.drop(df_clean[df_clean["Movie Name"] == "Valentine's Day Special"].index, inplace=True)
df_clean

,Movie Name,Language,IMDb Rating,Running Time,Year of Release,Maturity Rating
0,John Rambo,Tamil,7,1 h 26 min,2008,18+
1,American Pie,English,7,1 h 35 min,1999,18+
2,Bombshell,English,6.8,1 h 48 min,2019,18+
3,Love Birds,Tamil,5.1,2 h 40 min,1996,All
4,Hippi,Telugu,5,2 h 20 min,2019,18+
...,...,...,...,...,...,...
8121,Borat Subsequent Moviefilm,English,6.7,1 h 36 min,2020,18+
8122,Blackout,English,5.2,1 h 31 min,2014,18+
8123,Loafer,Hindi,6.5,2 h 3 min,1973,All
8124,Remember Me,English,7.1,1 h 52 min,2010,13+


In [ ]:
#Data types for columns were object type 
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5836 entries, 0 to 8126
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Movie Name       5836 non-null   object
 1   Language         5836 non-null   object
 2   IMDb Rating      5836 non-null   object
 3   Running Time     5836 non-null   object
 4   Year of Release  5836 non-null   object
 5   Maturity Rating  5836 non-null   object
dtypes: object(6)
memory usage: 319.2+ KB


In [ ]:
#Create a dataframe with only movies in English
df_english = df_clean[df_clean['Language'] == 'English']
df_english

,Movie Name,Language,IMDb Rating,Running Time,Year of Release,Maturity Rating
1,American Pie,English,7,1 h 35 min,1999,18+
2,Bombshell,English,6.8,1 h 48 min,2019,18+
13,Sealers: One Last Hunt,English,7.7,56min,2020,13+
15,Project Almanac,English,6.4,1 h 46 min,2015,13+
17,Ghost Stories,English,6.4,1 h 33 min,2018,18+
...,...,...,...,...,...,...
8107,Annabelle (2014),English,5.4,1 h 38 min,2014,18+
8115,Secretary (2002),English,7,1 h 51 min,2002,18+
8121,Borat Subsequent Moviefilm,English,6.7,1 h 36 min,2020,18+
8122,Blackout,English,5.2,1 h 31 min,2014,18+


In [ ]:
#Create a dataframe with maturity rating for all age groups
df_maturity_rating = df_clean[df_clean['Maturity Rating'] == 'All']
df_maturity_rating

,Movie Name,Language,IMDb Rating,Running Time,Year of Release,Maturity Rating
3,Love Birds,Tamil,5.1,2 h 40 min,1996,All
6,Ayyaa,Tamil,5.7,2 h 39 min,2005,All
10,Tum Se Achcha Kaun Hai,Hindi,4.5,2 h 21 min,2002,All
11,Swamy,Kannada,6,2 h 34 min,2005,All
12,Ekkees Tareekh Shubh Muhurat,Hindi,5.7,1 h 33 min,2018,All
...,...,...,...,...,...,...
8114,Pata Nahi Rabb Kehdeyan Rangan Ch Raazi,Punjabi,6.5,2 h 32 mins,2012,All
8117,Munna Bhai M.B.B.S.,Hindi,8.1,2 h 29 min,2003,All
8118,Badla,Bengali,6.6,2 h 29 min,2010,All
8120,Artist,Malayalam,7.1,1 h 47 min,2013,All


In [ ]:
language_df = df_clean.groupby("Language").count()
language_df

,Movie Name,IMDb Rating,Running Time,Year of Release,Maturity Rating
Language,,,,,
Bengali,529,529,529,529,529
English,1307,1307,1307,1307,1307
Gujarati,73,73,73,73,73
Hindi,1537,1537,1537,1537,1537
Kannada,252,252,252,252,252
Malayalam,433,433,433,433,433
Marathi,305,305,305,305,305
Punjabi,122,122,122,122,122
Tamil,586,586,586,586,586


In [ ]:
#Function that converts string types into integers in mins 
a = "2h"            #k[0-1] --> 2

#*** min****
b = "50min"         #a[0-4] --> 5 # good
c = "56 min"        #b[0-5] --> 6 # good
d = "1h 3min"       #c[0-6] --> 7 # good
e = "2h 52min"      #d[0-7] --> 8 # good
f = "2 h 8min"      #e[0-7] --> 8 # good
g = "2h 5 min"      #f[0-7] --> 8 # good
h = "3 h 28min"     #g[0-8] --> 9 # good
i = "1h 18 min"     #h[0-8] --> 9 # good
j = "2 h 9 min"     #h[0-8] --> 9 # good
k = "1 h 42 min"    #j[0-9] --> 10 # good

#*** mins ***
l = "32mins"         #l[0-5] --> 6 # good
m = "45 mins"        #m[0-6] --> 7 # good
n = "4h 5mins"       #n[0-7] --> 8 # good
o = "1h 21mins"      #o[0-8] --> 9 # good
p = "1 h 3mins"      #p[0-8] --> 9 # good
q = "3h 7 mins"      #q[0-8] --> 9 # good
r = "1 h 11mins"     #r[0-9] --> 10 # good
s = "2h 54 mins"     #s[0-9] --> 10 # good
t = "3 h 2 mins"     #t[0-9] --> 10 # good
u = "1 h 22 mins"    #u[0-10] --> 11 # good

# print("\nOriginal string a: " + a)
# print("Original string b: " + b)
# print("Original string c: " + c)
# print("Original string d: " + d)
# print("Original string e: " + e)
# print("Original string f: " + f)
# print("Original string g: " + g)
# print("Original string h: " + h)
# print("Original string i: " + i)
# print("Original string j: " + j)
# print("Original string k: " + k)

def getMinutes(x):
    temp_int = 0
    if(len(x) == 2):
        x = x.replace('h', '')
        temp_int = int(x)*60
        return temp_int
        # print("A in minutes: " + str(temp_int))

    elif(len(x) == 5):
        x = x.replace("min", "")
        return int(x)
        # print("B in minutes: " + str(x))
    
    elif (len(x) == 6):
        x = x.replace('m', ' ')
        s = x.split(' ')
        return int(s[0])
        # print("C then L in minutes: " + str(s[0]))
    
    elif(len(x) == 7):
        x = x.replace('m', ' ')
        x = x.replace('h','')
        s = x.split(' ')

        for value in range(len(s)-1):
            if(s[value] == ''):
                s.pop(value)

        if(len(s) == 3):
            temp_int = int(s[0])*60 + int(s[1])
            return temp_int
            # print("D in minutes: " + str(temp_int))
        
        elif(len(s) == 2):
            temp_int = int(s[0])
            return temp_int
            # print('M in minutes: ' + str(temp_int))

    # e = "2h 52min"
    # f = "2 h 8min"
    # g = "2h 5 min"
    # n = "4h 5mins"
    elif(len(x) == 8):
        x = x.replace('h', '')
        x = x.replace(' min','')
        x = x.replace('mins','')
        x = x.replace('min','')
        s = x.split(' ')

        for value in range(len(s)-1):
            if(s[value] == ''):
                s.pop(value)
        
        temp_int = int(s[0])*60 + int(s[1])
        return temp_int
        # print("E, F, G then N in minutes: " + str(temp_int))
    
    # h = "3 h 28min"
    # i = "1h 18 min"
    # j = "2 h 9 min"
    # o = "1h 21mins"
    # p = "1 h 3mins"
    # q = "3h 7 mins"
    elif(len(x) == 9):
        x = x.replace('h', '')
        x = x.replace('mins','')
        x = x.replace(' min','')
        x = x.replace('min','')
        s = x.split(' ')

        for value in range(len(s)-1):
            if(s[value] == ''):
                s.pop(value)

        temp_int = int(s[0])*60 + int(s[1])
        return temp_int
        # print("H, I, J, O, P, and Q in minutes: " + str(temp_int))

    # k = "1 h 42 min"
    # r = "1 h 11mins"
    # s = "2h 54 mins"
    # t = "3 h 2 mins"
    elif(len(x) == 10):
        x = x.replace('h', '')
        x = x.replace('mins','')
        x = x.replace(' min','')
        s = x.split(' ')

        for value in range(len(s)-1):
            if(s[value] == ''):
                s.pop(value)

        temp_int = int(s[0])*60 + int(s[1])
        return temp_int
        # print("K, R, S, then T in minutes: " + str(temp_int))
   
    # u = "1 h 22 mins"
    elif(len(x) == 11):
        x = x.replace('h', '')
        x = x.replace('mins','')
        s = x.split(' ')

        for value in range(len(s)-1):
            if(s[value] == ''):
                s.pop(value)

        temp_int = int(s[0])*60 + int(s[1])
        return temp_int
        # print("U in minutes: " + str(temp_int))

# length of 2
    # a = "2h"
getMinutes(a)

# length of 5
    # b = "50min"
getMinutes(b)

# length of 6
    # c = "56 min"
    # l = "32mins"
getMinutes(c)
getMinutes(l)

# length of 7
    # d = "1h 3min"
    # m = "45 mins"
getMinutes(d)
getMinutes(m)

# length of 8
    # e = "2h 52min"
    # f = "2 h 8min"
    # g = "2h 5 min"
    # n = "4h 5mins"
getMinutes(e)
getMinutes(f)
getMinutes(g)
getMinutes(n)

# length of 9
    # h = "3 h 28min"
    # i = "1h 18 min"
    # j = "2 h 9 min"
    # o = "1h 21mins"
    # p = "1 h 3mins"
    # q = "3h 7 mins"
getMinutes(h)
getMinutes(i)
getMinutes(j)
getMinutes(o)
getMinutes(p)
getMinutes(q)

# length of 10
    # k = "1 h 42 min"
    # r = "1 h 11mins"
    # s = "2h 54 mins"
    # t = "3 h 2 mins"
getMinutes(k)
getMinutes(r)
getMinutes(s)
getMinutes(t)

# length of 11
    # u = "1 h 22 mins"
getMinutes(u)

82

In [ ]:
#Calls getMinutes function to convert running time 
df_clean["minutes"] = df_clean["Running Time"].map(getMinutes)
df_clean

,Movie Name,Language,IMDb Rating,Running Time,Year of Release,Maturity Rating,minutes
0,John Rambo,Tamil,7,1 h 26 min,2008,18+,86.0
1,American Pie,English,7,1 h 35 min,1999,18+,95.0
2,Bombshell,English,6.8,1 h 48 min,2019,18+,108.0
3,Love Birds,Tamil,5.1,2 h 40 min,1996,All,160.0
4,Hippi,Telugu,5,2 h 20 min,2019,18+,140.0
...,...,...,...,...,...,...,...
8121,Borat Subsequent Moviefilm,English,6.7,1 h 36 min,2020,18+,96.0
8122,Blackout,English,5.2,1 h 31 min,2014,18+,91.0
8123,Loafer,Hindi,6.5,2 h 3 min,1973,All,123.0
8124,Remember Me,English,7.1,1 h 52 min,2010,13+,112.0


In [ ]:
df_clean.to_csv('/content/gdrive/My Drive/Colab Datasets/update clean amazon movies.csv', index = True)